### 計算機程式專案報告 --- 分辨10種動物

### 專案分工

#### 105306047 資管四 張鎧 編輯程式碼、註解
#### 105306015 資管四 余柏霖 收集資料、整理資料
#### 105303090 會計四 洪瑞伸 編輯程式碼

### 問題發想與困難

#### 對大人來說辨別物種不是一件難事，但對於學齡前兒童可能是一大難關，故我們想透過這個程式幫助小朋友認識物種，為了方便小孩子吸收，我們先從10種物種開始，之後有想過要增加辨識色情圖片的功能，也有找到色情圖片的資料集，但後來發現要把兩個功能結合有點困難，我們最後還是決定自己再做一個辨識動物的程式。

### 資料來源：https://www.kaggle.com/alessiocorrado99/animals10

### import 和 讀入數據集

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
activated = False
if activated:
    data = []
    labels = []
    img_size = (50, 50)

    img_folder_path = 'img'
    folders = os.listdir(img_folder_path)
    img_num = 1000

    for i in range(len(folders)):
        folder_name = folders[i]
        folder_path = img_folder_path + '/' + folder_name
        folder = os.listdir(folder_path)
        for img_name in folder[:img_num]:
            img_path = folder_path + '/' + img_name
            img = cv2.imread(img_path)
            img_from_array = Image.fromarray(img, 'RGB')
            img_resized = img_from_array.resize(img_size)
            data.append(np.array(img_resized))
            labels.append(i)

    animals = np.array(data)
    labels = np.array(labels)
    np.save("animals", animals)
    np.save("labels", labels)

In [3]:
animals = np.load("animals.npy")
labels = np.load("labels.npy")

data_length = len(animals)
classes_num = len(np.unique(labels))

order = np.arange(data_length)
np.random.shuffle(order)

animals = animals[order]
labels = labels[order]


In [4]:
ratio = 0.1
(x_train, x_test) = animals[int(ratio * data_length) : ], animals[ : int(ratio * data_length)]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

train_length = len(x_train)
test_length = len(x_test)

(y_train, y_test) = labels[int(ratio * data_length) : ], labels[ : int(ratio*data_length)]

### one-hot-encoding

In [9]:
# one-hot-encoding
import keras
from keras.utils import np_utils

y_train = keras.utils.to_categorical(y_train, classes_num)
y_test = keras.utils.to_categorical(y_test, classes_num)

Using TensorFlow backend.


### 建造捲積神經網路 CNN

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding="same", activation="relu", input_shape=(50, 50 , 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(classes_num, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
____________________

### 訓練模型

In [11]:
model.fit(x_train, y_train, batch_size=50, epochs=20, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





9000/9000 [==============================] - 14s 2ms/step - loss: 2.0429 - acc: 0.2524
Epoch 2/20
9000/9000 [==============================] - 13s 1ms/step - loss: 1.7051 - acc: 0.3997
Epoch 3/20
9000/9000 [==============================] - 13s 1ms/step - loss: 1.5222 - acc: 0.4691
Epoch 4/20
9000/9000 [==============================] - 13s 1ms/step - loss: 1.3707 - acc: 0.5287
Epoch 5/20
9000/9000 [==============================] - 13s 1ms/step - loss: 1.2654 - acc: 0.5591
Epoch 6/20
9000/9000 [==============================] - 13s 1ms/step - loss: 1.1539 - acc: 0.6013
Epoch 7/20
9000/9000 [==============================] - 14s 2ms/step - loss: 1.0959 - acc: 0.6261
Epoch 8/20
9000/9000 [==============================] - 13s 1ms/step - loss: 0.9891 - acc: 0.6609
Epoch 9/20
9000/9000 [==============================] - 14s 2ms/step - loss: 0.9001 - acc: 0.6913
Epoch 10/20
9000

In [28]:
score = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test accuracy:', score[1])

1000/1000 [==============================] - 1s 972us/step

 Test accuracy: 0.602


### 建立及測試功能

In [42]:
from ipywidgets import interact_manual

def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)

def get_animal_name(label):
    label_dict = {'0': 'butterfly', '1': 'cat', '2': 'chicken', '3': 'cow', '4': 'dog', '5': 'elephant',\
     '6': 'horse', '7': 'sheep', '8': 'spider', '9': 'squirrel'}
    res = label_dict[str(label)]
    return res
    
def predict_animal(file):
    print("Predicting .................................")
    ar = convert_to_array(file)
    ar = ar/255
    label = 1
    a = []
    a.append(ar)
    a = np.array(a)
    score = model.predict(a,verbose=1)
    print(score)
    label_index = np.argmax(score)
    print(label_index)
    acc = np.max(score)
    animal = get_animal_name(label_index)
    print(animal)
    print("Animal : " + animal + "\nwith accuracy =    " + str(acc))

def demo_test(n):
    i = x_test[n]
    plt.imshow(i)
    i = i.reshape(1, i.shape[0], i.shape[1], i.shape[2])
    score = model.predict(i,verbose=0)
    label_index = np.argmax(score)
    acc = np.max(score)
    animal = get_animal_name(label_index)
    print("Animal : " + animal + "\nwith accuracy =    " + str(acc))


In [43]:
interact_manual(demo_test, n = (0, len(x_test)-1));

interactive(children=(IntSlider(value=499, description='n', max=999), Button(description='Run Interact', style…

In [44]:
predict_animal("test/butterfly.jpeg")

Predicting .................................
1/1 [==============================] - 0s 3ms/step
[[7.7545065e-01 1.6267444e-01 1.1766575e-02 3.1154777e-08 3.6069020e-03
  1.1624001e-05 5.9630292e-09 6.7512769e-07 4.6482239e-02 6.9391672e-06]]
0
butterfly
Animal : butterfly
with accuracy =    0.77545065


### 得到7成7的準確度

### 結論： 
#### 經過這次的實作之後，我們發現在執行一個專案必須考慮到非常多的細節，只要一個小地方出錯，牽一髮而動全身會影響到整個測試的結果，最後的準確度雖不到非常精準，但仍有一定的準確率，在之後我們也會繼續嘗試其他方式或調整參數以提高準確率。

### 實作心得：
#### 我們這組的組成儘管有資管系的同學，但我們在實作的過程中還是遇到了一些困難，譬如會計系的同學提到說他之前從沒接觸過程式相關的課程及學習，因此我們在建立神經網路的過程，還是盡量依照炎龍老師課堂上的示範跟著操作，最後至少在測試資料的準確度也得到不錯的效果，而前面整理資料的過程，雖然只有十種動物，但整體的資料量依舊相當大，因此在這次的專案中，我們也實際體會到整理資料並選擇資料量的大小的重要性。